<a href="https://colab.research.google.com/github/Daksh024/NSP/blob/Colab/smallCorpus_20epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForMaskedLM, AdamW

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:

# Load and tokenize text corpus from a file
corpus_file_path = "/content/drive/MyDrive/smallCorpus.txt"
with open(corpus_file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()


In [4]:

data = []
for line in lines:
    tokens = tokenizer.tokenize(line)
    for i in range(len(tokens) - 1):
        data.append((tokens[i], tokens[i+1]))


In [5]:
data

[('ल', '##खन'),
 ('##खन', '##ऊ'),
 ('##ऊ', 'का'),
 ('का', 'एका'),
 ('एका', '##ना'),
 ('##ना', 'स'),
 ('स', '##्ट'),
 ('##्ट', '##ेड'),
 ('##ेड', '##ियम'),
 ('##ियम', 'अपना'),
 ('अपना', 'पहला'),
 ('पहला', 'म'),
 ('म', '##ैच'),
 ('##ैच', 'आय'),
 ('आय', '##ोजित'),
 ('##ोजित', 'कर'),
 ('कर', 'रहा'),
 ('रहा', 'है'),
 ('है', '।'),
 ('।', 'इस'),
 ('इस', 'ल'),
 ('ल', '##ि'),
 ('##ि', '##हा'),
 ('##हा', '##ज'),
 ('##ज', 'से'),
 ('से', 'प'),
 ('प', '##ि'),
 ('##ि', '##च'),
 ('##च', 'एक'),
 ('एक', '##द'),
 ('##द', '##म'),
 ('##म', 'नई'),
 ('नई', 'हो'),
 ('हो', '##गी'),
 ('##गी', 'और'),
 ('और', 'किसी'),
 ('किसी', 'को'),
 ('को', 'इसके'),
 ('इसके', 'बारे'),
 ('बारे', 'में'),
 ('में', 'कोई'),
 ('कोई', 'अ'),
 ('अ', '##ंद'),
 ('##ंद', '##ाज'),
 ('##ाज', '##ा'),
 ('##ा', 'नहीं'),
 ('नहीं', 'होगा'),
 ('होगा', '।'),
 ('।', 'ह'),
 ('ह', '##ल'),
 ('##ल', '##्क'),
 ('##्क', '##ी'),
 ('##ी', 'स'),
 ('स', '##र'),
 ('##र', '##्दी'),
 ('##्दी', 'रहे'),
 ('रहे', '##गी'),
 ('##गी', 'इसलिए'),
 ('इसलिए', 'ओ'),
 ('ओ'

In [6]:

# Define custom dataset
class NextWordDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_token, target_token = self.data[idx]
        input_ids = self.tokenizer.convert_tokens_to_ids(input_token)
        target_token_id = self.tokenizer.convert_tokens_to_ids(target_token)
        return torch.tensor(input_ids), torch.tensor(target_token_id)

# Create DataLoader for the dataset
train_dataset = NextWordDataset(data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

# # Fine-tuning loop
# num_epochs = 10
# for batch in train_dataloader:
#     optimizer.zero_grad()
#     input_ids, target_ids = batch

#     # Ensure input_ids is a 2D tensor
#     input_ids = input_ids.unsqueeze(0) if input_ids.dim() == 1 else input_ids

#     outputs = model(input_ids)[0]
#     loss = loss_fn(outputs.view(-1, outputs.shape[-1]), target_ids)
#     loss.backward()
#     optimizer.step()

import time

num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Start the timer
start_time = time.time()

for epoch in range(num_epochs):
    e_start_time = time.time()

    for batch in train_dataloader:

        b_start_time = time.time()

        optimizer.zero_grad()
        input_ids, target_ids = batch

        # Move data to GPU
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)

        # print(input_ids)

        # Ensure input_ids is a 2D tensor
        input_ids = input_ids.unsqueeze(0) if input_ids.dim() == 1 else input_ids

        outputs = model(input_ids)[0]

        # Flatten both outputs and targets
        outputs_flat = outputs.view(-1, outputs.shape[-1])
        target_ids_flat = target_ids.view(-1)

        loss = loss_fn(outputs_flat, target_ids_flat)
        loss.backward()
        optimizer.step()

        b_end_time = time.time()

        b_elapsed_time = b_end_time - b_start_time
        # print(f"batch took {b_elapsed_time:.6f} seconds")

    e_end_time = time.time()

    print("Loss at end of the epoch is {}".format(loss))

    e_elapsed_time = e_end_time - e_start_time
    print(f"epoch took {e_elapsed_time:.6f} seconds")

    # print("loss after each epoch{}".format(loss))

    if epoch%10 == 0:
        model.save_pretrained("/content/drive/MyDrive/fine_tuned_bert_model_{}".format(epoch))
        tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned_bert_model_{}".format(epoch))

end_time = time.time()


Loss at end of the epoch is 4.3910136222839355
epoch took 713.985845 seconds
Loss at end of the epoch is 4.719020843505859
epoch took 709.339197 seconds
Loss at end of the epoch is 4.495241641998291
epoch took 708.454062 seconds
Loss at end of the epoch is 4.579504489898682
epoch took 706.680087 seconds
Loss at end of the epoch is 4.374192714691162
epoch took 704.016685 seconds
Loss at end of the epoch is 3.5087485313415527
epoch took 694.721329 seconds
Loss at end of the epoch is 4.478335857391357
epoch took 699.067230 seconds
Loss at end of the epoch is 3.886723279953003
epoch took 697.440991 seconds
Loss at end of the epoch is 4.648687362670898
epoch took 696.243423 seconds
Loss at end of the epoch is 3.698028564453125
epoch took 695.213234 seconds
Loss at end of the epoch is 5.052340984344482
epoch took 699.856698 seconds
Loss at end of the epoch is 4.364907264709473
epoch took 708.078153 seconds
Loss at end of the epoch is 4.9356231689453125
epoch took 717.071473 seconds
Loss at e

In [ ]:
loss

tensor(3.5059, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
elapsed_time = end_time - start_time
print(f"total time took {elapsed_time:.6f} seconds")

In [ ]:
model.save_pretrained("fine_tuned_bert_model")
tokenizer.save_pretrained("fine_tuned_bert_model")

In [ ]:
# Load the fine-tuned model
fine_tuned_model = BertForMaskedLM.from_pretrained("fine_tuned_bert_model")



In [3]:
model.save_pretrained("/content/drive/MyDrive/fine_tuned_bert_model_20")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned_bert_model_20")

# /content/drive/MyDrive/fine_tuned_bert_model_20

('/content/drive/MyDrive/fine_tuned_bert_model_20/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_bert_model_20/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_bert_model_20/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_bert_model_20/added_tokens.json')

In [4]:
fine_tuned_model = BertForMaskedLM.from_pretrained("/content/drive/MyDrive/fine_tuned_bert_model_20")

In [ ]:
# # Inference
# input_text = "मैं"
# input_ids = tokenizer.encode(input_text, add_special_tokens=True)
# with torch.no_grad():
#     outputs = fine_tuned_model(torch.tensor(input_ids).unsqueeze(0))
#     predicted_token_id = torch.argmax(outputs[0, -1]).item()
#     predicted_word = tokenizer.convert_ids_to_tokens(predicted_token_id)

# print("Predicted next word:", predicted_word)

# print(outputs.logits)

# Input text
input_text = "मैं"

# Encode input text and perform inference
input_ids = tokenizer.encode(input_text, add_special_tokens=True)
with torch.no_grad():
    outputs = fine_tuned_model(torch.tensor(input_ids).unsqueeze(0))
    print(outputs.logits)
    predicted_token_id = torch.argmax(outputs.logits[0, -1]).item()
    predicted_word = tokenizer.convert_ids_to_tokens(predicted_token_id)

print("Predicted word:", predicted_word)

In [5]:
# Input text
input_text = "मैं"

# Encode input text
input_ids = tokenizer.encode(input_text, add_special_tokens=True)

generated_ids = input_ids.copy()
with torch.no_grad():
    for _ in range(50):
        input_tensor = torch.tensor(generated_ids).unsqueeze(0)
        outputs = fine_tuned_model(input_tensor)
        predicted_token_id = torch.argmax(outputs.logits[0, -1]).item()
        generated_ids.append(predicted_token_id)
        if predicted_token_id == tokenizer.sep_token_id:
            break

# Convert the generated IDs to words
generated_words = tokenizer.convert_ids_to_tokens(generated_ids)

# Print the generated sequence
generated_sequence = " ".join(generated_words)
# print("Generated sequence:", generated_sequence)
print(tokenizer.convert_tokens_to_string(generated_words))

[CLS] मैं [SEP] , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,


In [ ]:
from transformers import BertTokenizer

# Load the tokenizer for a multilingual BERT model
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Hindi input text
hindi_text = "मैंने खाना खाया"

# Tokenize the Hindi text
tokens = tokenizer.tokenize(hindi_text)
print(tokens)

# Convert tokens to IDs
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(input_ids)


['म', '##ैं', '##ने', 'खान', '##ा', 'खा', '##या']
[889, 99007, 13466, 101415, 11208, 64566, 15168]


In [ ]:
from transformers import BertTokenizer

# Load the tokenizer for a multilingual BERT model
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# List of token IDs for Hindi words
hindi_word_ids = [tokenizer.convert_tokens_to_ids(token) for token in ['म', '##ैं', '##ने', 'खान', '##ा', 'खा', '##या']]

# Convert token IDs back to words
hindi_words = tokenizer.convert_ids_to_tokens(hindi_word_ids)
print(hindi_words)

['म', '##ैं', '##ने', 'खान', '##ा', 'खा', '##या']


In [ ]:
tokenizer.convert_tokens_to_string(['म', '##ैं', '##ने', 'खान', '##ा', 'खा', '##या'])

'मैंने खाना खाया'